In [ ]:
'pilot_total_ltm_stats/correct_wm_ltm_p_pilot_total.csv'

In [1]:
library(lme4)
library(simr)
library(dplyr)
library(broom)
install.packages("sjPlot")
library(sjPlot)
install.packages("broom.mixed")
library(broom.mixed)
library(htmlTable)
library(dplyr)
library(stringr)
# library(gt)
library(car)
install.packages("stargazer")
library(stargazer)
# First install and load lmerTest
install.packages("lmerTest")
library(lmerTest)

Loading required package: Matrix


Attaching package: ‘simr’


The following object is masked from ‘package:lme4’:

    getData



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpu6ny1y/downloaded_packages

The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpu6ny1y/downloaded_packages



Attaching package: ‘stringr’


The following object is masked from ‘package:simr’:

    fixed


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpu6ny1y/downloaded_packages



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 





The downloaded binary packages are in
	/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T//Rtmpu6ny1y/downloaded_packages



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [29]:
# file_path_total <- 'pilot_total_ltm_stats/correct_wm_ltm_p_pilot_total.csv'
# file_path_5 <- 'pilot5_ltm_stats/correct_wm_ltm_p_pilot5.csv'
# file_path_6 <- 'pilot6_ltm_stats/correct_wm_ltm_p_pilot6.csv'
# file_path_total <- 'enhanced_df.csv'
df_total <- read.csv(file_path_total) 
df5 <- read.csv(file_path_5) 
df6 <- read.csv(file_path_6) 



In [27]:
run_models <- function(df) {

    
    acc_rel <- glmer(recall_answ ~   p_values_v2_flipped_ltm + p_values_it_flipped_ltm +
                        + reliability_binary_z *  attend_binary_z  + tested_binary_z + it_pos_neg_ltm + v2_pos_neg_ltm + (1 | participant), 
                      data = df, family = binomial,control = glmerControl(optimizer = "bobyqa",
                                            optCtrl = list(maxfun = 500000),
                                            calc.derivs = FALSE))


return(list(
    acc_rel = acc_rel
  ))
}

In [28]:
results_old_stims <- run_models(df6)
results_new_stims <- run_models(df5)
results_total <- run_models(df_total)

ERROR: Error in eval(predvars, data, env): object 'p_values_v2_flipped_ltm' not found


In [22]:
extract_significance <- function(model_summary) {
  # For glmer models
  if ("coefficients" %in% names(model_summary)) {
    # Check more carefully for z value vs t value columns
    coef_table <- model_summary$coefficients
    var_names <- rownames(coef_table)
    
    # Check which p-value column exists
    if ("Pr(>|z|)" %in% colnames(coef_table)) {
      p_values <- coef_table[, "Pr(>|z|)"]
    } else if ("Pr(>|t|)" %in% colnames(coef_table)) {
      p_values <- coef_table[, "Pr(>|t|)"]
    } else {
      # If there's a z or t value, calculate p-values manually
      if ("z value" %in% colnames(coef_table)) {
        z_values <- coef_table[, "z value"]
        p_values <- 2 * pnorm(abs(z_values), lower.tail = FALSE)
      } else if ("t value" %in% colnames(coef_table)) {
        t_values <- coef_table[, "t value"]
        # Using a large df for approximation
        p_values <- 2 * pt(abs(t_values), df = 100, lower.tail = FALSE)
      } else {
        stop("Cannot find p-values or z/t values in coefficients table")
      }
    }
  } else {
    # More detailed debug info
    cat("Available names in model summary:\n")
    print(names(model_summary))
    
    # Try to access fixed effects via other means
    if ("fixed.effects" %in% names(model_summary)) {
      var_names <- names(model_summary$fixed.effects)
      # Manually calculate p-values using fixed effects and std errors
      # This is just a placeholder - would need to be adapted to actual structure
      stop("Found fixed.effects but need appropriate calculation of p-values")
    } else {
      stop("Could not extract coefficients from model summary")
    }
  }
  
  # Calculate significance stars
  significance <- sapply(p_values, function(p) {
    if (is.na(p)) return("")
    if (p < 0.001) return("***")
    if (p < 0.01) return("**")
    if (p < 0.05) return("*")
    if (p < 0.1) return(".")
    return("")
  })
  
  return(data.frame(
    variable = var_names,
    p_value = p_values,
    significance = significance,
    stringsAsFactors = FALSE
  ))
}


In [23]:
compare_models <- function(model_list, model_names, 
                          include_p_values = FALSE,
                          only_significant = TRUE,
                          significance_level = 0.1) {
  
  sig_list <- lapply(model_list, extract_significance)
  names(sig_list) <- model_names
  
  all_vars <- unique(unlist(lapply(sig_list, function(x) x$variable)))
  
  result_df <- data.frame(Variable = all_vars, stringsAsFactors = FALSE)
  
  # First, determine which rows have significance in any model
  has_significance_by_var <- sapply(all_vars, function(var) {
    any_significant <- FALSE
    for(model_name in model_names) {
      model_sig <- sig_list[[model_name]]
      idx <- which(model_sig$variable == var)
      if(length(idx) > 0) {
        stars <- model_sig$significance[idx]
        if(stars != "") {
          any_significant <- TRUE
          break
        }
      }
    }
    return(any_significant)
  })
  
  # Now populate the data frame with all p-values for rows with any significance
  for(model_name in model_names) {
    model_sig <- sig_list[[model_name]]
    result_df[[model_name]] <- sapply(all_vars, function(var) {
      idx <- which(model_sig$variable == var)
      if(length(idx) == 0) return("")
      
      p_val <- model_sig$p_value[idx]
      stars <- model_sig$significance[idx]
      
      # Always show p-value if this variable has significance in any model
      if(has_significance_by_var[var] || !only_significant) {
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      } else if(stars != "") {
        # Show significance stars with p-value for significant results
        return(paste0(stars, " (p=", sprintf("%.4f", p_val), ")"))
      } else {
        # For non-significant results in rows without any significance
        return("")
      }
    })
  }
  
  # Filter rows if only_significant is TRUE
  if(only_significant) {
    result_df <- result_df[has_significance_by_var, ]
  }
  
  # Adjust HTML table styling to make first column wider
  html_table <- htmlTable(
    result_df,
    caption = "Significance Comparison Across Models",
    rnames = FALSE,
    css.cell = "padding: 8px;",
    css.table = "border-collapse: collapse; width: 100%; font-family: Arial, sans-serif;",
    css.header = "background-color: #f2f2f2; font-weight: bold; border: 1px solid #dddddd;",
    css.rgroup = "border-bottom: 1px solid #dddddd;",
    css.tspanner = "font-weight: bold; text-align: center; border-bottom: 2px solid #dddddd;",
    css.class = "table-hover"
  )
  
  # Add additional CSS to make the first column wider
  html_table <- gsub("<table", "<style>td:first-child, th:first-child { min-width: 300px; max-width: 500px; word-wrap: break-word; }</style><table", html_table)
  
  return(html_table)
}

In [15]:




acc_rel_old <- summary(results_old_stims$acc_rel)
acc_rel_new <- summary(results_new_stims$acc_rel)
acc_rel_total <- summary(results_total$acc_rel)

comparison_table_acc_rel <- compare_models(
  model_list = list(acc_rel_old, acc_rel_new, acc_rel_total),
  model_names = c("Old Stimuli", "New Stimuli", "Total"),
  include_p_values = TRUE,
  only_significant = TRUE
)
writeLines(comparison_table_acc_rel, "comparison_table_wm_acc_rel.html")

# Display all tables
IRdisplay::display_html(comparison_table_acc_rel)


Significance Comparison Across Models 
 
 Variable 
 Old Stimuli 
 New Stimuli 
 Total 
 
 
 
 
 (Intercept) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 reliability_binary_z 
 (p=0.2701) 
 * (p=0.0287) 
 * (p=0.0202) 
 
 
 attend_binary_z 
 *** (p=0.0000) 
 *** (p=0.0000) 
 *** (p=0.0000) 
 
 
 reliability_binary_z:attend_binary_z 
 *** (p=0.0000) 
 ** (p=0.0066) 
 *** (p=0.0000)

In [16]:
acc_rel_old

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
recall_answ ~ reliability_binary_z * attend_binary_z + (1 | participant)
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
  7903.4   7938.1  -3946.7   7893.4     7675 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1283  0.2138  0.4188  0.5832  1.3369 

Random effects:
 Groups      Name        Variance Std.Dev.
 participant (Intercept) 0.5166   0.7188  
Number of obs: 7680, groups:  participant, 134

Fixed effects:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           1.28948    0.06934  18.597  < 2e-16 ***
reliability_binary_z                  0.03170    0.02874   1.103     0.27    
attend_binary_z                      -0.36297    0.02882 -12.593  < 2e-16 ***
reliability_binary_z:attend_binary_z -0.12396    0.02875  -4.312 

In [17]:
acc_rel_new

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
recall_answ ~ reliability_binary_z * attend_binary_z + (1 | participant)
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
  7133.0   7167.5  -3561.5   7123.0     7328 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4237  0.2677  0.3992  0.5432  1.5217 

Random effects:
 Groups      Name        Variance Std.Dev.
 participant (Intercept) 0.5031   0.7093  
Number of obs: 7333, groups:  participant, 134

Fixed effects:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           1.41588    0.06936  20.413  < 2e-16 ***
reliability_binary_z                  0.06709    0.03067   2.187  0.02871 *  
attend_binary_z                      -0.37160    0.03074 -12.089  < 2e-16 ***
reliability_binary_z:attend_binary_z -0.08305    0.03055  -2.718 

In [18]:
acc_rel_total

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
recall_answ ~ reliability_binary_z * attend_binary_z + (1 | participant)
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
 15029.9  15068.0  -7510.0  15019.9    15008 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4036  0.2473  0.4084  0.5591  1.5626 

Random effects:
 Groups      Name        Variance Std.Dev.
 participant (Intercept) 0.5141   0.717   
Number of obs: 15013, groups:  participant, 268

Fixed effects:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           1.35222    0.04919  27.488  < 2e-16 ***
reliability_binary_z                  0.04870    0.02097   2.322   0.0202 *  
attend_binary_z                      -0.36721    0.02102 -17.466  < 2e-16 ***
reliability_binary_z:attend_binary_z -0.10473    0.02093  -5.003